In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
from src.bot.util.features import clean_features
from src.bot.util.model import ModelPool, MergeModel
from src.bot.util.data import *
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

data_dir = 'data/final/';
model_dir = 'src/model/final/';

In [5]:
X, y = stack_Xy(data_dir, 7)

In [2]:
X = pd.read_pickle(data_dir + 'X.pickle')
y = pd.read_pickle(data_dir + 'y.pickle')
names = y['private_bot_name'].value_counts().index.sort_values().tolist()
names

['KillFish', 'dannyace', 'fcll']

# TRAIN MERGE

In [6]:
def train_call_model():
    model_call_pool = ModelPool(len(names), model_dir + 'pool/call/')
    y_call = y['private_bot_action'].replace({'FOLD': 0, 'CALL': 1, 'RAISE': 1})
    X_call = model_call_pool.predict(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_call, y_call, test_size=0.1, random_state=1234)
    
    model = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=2, thread_count=4,
                                   verbose=True, use_best_model=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test))
    model.save_model(model_dir + 'call.model')

train_call_model()

Borders generated
0:	learn 0.605210514	test 0.6034840339	bestTest 0.6034840339		total: 144ms	remaining: 28.6s
1:	learn 0.5390684702	test 0.535776936	bestTest 0.535776936		total: 184ms	remaining: 18.2s
2:	learn 0.4749650363	test 0.4701498358	bestTest 0.4701498358		total: 218ms	remaining: 14.3s
3:	learn 0.4265717584	test 0.4209652143	bestTest 0.4209652143		total: 252ms	remaining: 12.3s
4:	learn 0.3992374768	test 0.392653052	bestTest 0.392653052		total: 286ms	remaining: 11.2s
5:	learn 0.3720572893	test 0.3644715542	bestTest 0.3644715542		total: 332ms	remaining: 10.7s
6:	learn 0.3564106135	test 0.3480660211	bestTest 0.3480660211		total: 370ms	remaining: 10.2s
7:	learn 0.3399500869	test 0.3307964629	bestTest 0.3307964629		total: 403ms	remaining: 9.68s
8:	learn 0.3263682411	test 0.3169388065	bestTest 0.3169388065		total: 438ms	remaining: 9.29s
9:	learn 0.3166301859	test 0.3069657718	bestTest 0.3069657718		total: 471ms	remaining: 8.95s
10:	learn 0.3107191034	test 0.3006033267	bestTest 0.30060

93:	learn 0.2753182207	test 0.2623167136	bestTest 0.2623167136		total: 3.29s	remaining: 3.71s
94:	learn 0.275315792	test 0.2623158177	bestTest 0.2623158177		total: 3.32s	remaining: 3.67s
95:	learn 0.2752971682	test 0.2622987289	bestTest 0.2622987289		total: 3.36s	remaining: 3.64s
96:	learn 0.2752820464	test 0.2622847654	bestTest 0.2622847654		total: 3.39s	remaining: 3.6s
97:	learn 0.2752795702	test 0.2622840351	bestTest 0.2622840351		total: 3.42s	remaining: 3.56s
98:	learn 0.2752775675	test 0.2622835961	bestTest 0.2622835961		total: 3.46s	remaining: 3.53s
99:	learn 0.2752670283	test 0.2622980522	bestTest 0.2622835961		total: 3.49s	remaining: 3.49s
100:	learn 0.2752534538	test 0.2622856276	bestTest 0.2622835961		total: 3.52s	remaining: 3.45s
101:	learn 0.2752516108	test 0.2622852624	bestTest 0.2622835961		total: 3.56s	remaining: 3.42s
102:	learn 0.2752403889	test 0.26227512	bestTest 0.26227512		total: 3.59s	remaining: 3.38s
103:	learn 0.2752403758	test 0.2622744177	bestTest 0.2622744177

184:	learn 0.2750677352	test 0.2624076727	bestTest 0.2622650943		total: 6.29s	remaining: 510ms
185:	learn 0.2750677345	test 0.2624075767	bestTest 0.2622650943		total: 6.33s	remaining: 476ms
186:	learn 0.2750675583	test 0.2624078082	bestTest 0.2622650943		total: 6.36s	remaining: 442ms
187:	learn 0.2750654386	test 0.2624052868	bestTest 0.2622650943		total: 6.39s	remaining: 408ms
188:	learn 0.2750654375	test 0.2624051543	bestTest 0.2622650943		total: 6.42s	remaining: 374ms
189:	learn 0.275065037	test 0.2624094827	bestTest 0.2622650943		total: 6.45s	remaining: 340ms
190:	learn 0.2750650362	test 0.2624093764	bestTest 0.2622650943		total: 6.49s	remaining: 306ms
191:	learn 0.2750650356	test 0.2624092808	bestTest 0.2622650943		total: 6.52s	remaining: 272ms
192:	learn 0.2750650351	test 0.2624091947	bestTest 0.2622650943		total: 6.56s	remaining: 238ms
193:	learn 0.2750650313	test 0.2624088004	bestTest 0.2622650943		total: 6.59s	remaining: 204ms
194:	learn 0.2750647066	test 0.2624127585	bestTest 

In [7]:
def train_raise_model():
    model_raise_pool = ModelPool(len(names), model_dir + 'pool/raise/')
    idx = y['private_bot_action'] != 'FOLD'
    y_raise = y[idx]['private_bot_action'].replace({'CALL': 0, 'RAISE': 1})
    X_raise = model_raise_pool.predict(X[idx])
    
    X_train, X_test, y_train, y_test = train_test_split(X_raise, y_raise, test_size=0.1, random_state=1234)
    
    model = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=2, thread_count=4,
                                   verbose=True, use_best_model=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test))
    model.save_model(model_dir + 'raise.model')

train_raise_model()

Borders generated
0:	learn 0.5917510347	test 0.5928738997	bestTest 0.5928738997		total: 446ms	remaining: 1m 28s
1:	learn 0.5173994022	test 0.5193508238	bestTest 0.5193508238		total: 474ms	remaining: 47s
2:	learn 0.4655223855	test 0.4679732957	bestTest 0.4679732957		total: 501ms	remaining: 32.9s
3:	learn 0.4247946251	test 0.4276871345	bestTest 0.4276871345		total: 528ms	remaining: 25.9s
4:	learn 0.3950398506	test 0.3982679995	bestTest 0.3982679995		total: 555ms	remaining: 21.6s
5:	learn 0.3739046213	test 0.3772632052	bestTest 0.3772632052		total: 581ms	remaining: 18.8s
6:	learn 0.3581731871	test 0.3616025789	bestTest 0.3616025789		total: 608ms	remaining: 16.8s
7:	learn 0.3463784306	test 0.3498391857	bestTest 0.3498391857		total: 640ms	remaining: 15.4s
8:	learn 0.3400325321	test 0.3436599355	bestTest 0.3436599355		total: 668ms	remaining: 14.2s
9:	learn 0.3308882402	test 0.3346710197	bestTest 0.3346710197		total: 694ms	remaining: 13.2s
10:	learn 0.3269662065	test 0.3309084486	bestTest 0.3

90:	learn 0.2973918009	test 0.3029316079	bestTest 0.3028047413		total: 3.02s	remaining: 3.62s
91:	learn 0.2973917943	test 0.3029313933	bestTest 0.3028047413		total: 3.05s	remaining: 3.58s
92:	learn 0.297391789	test 0.3029312009	bestTest 0.3028047413		total: 3.08s	remaining: 3.54s
93:	learn 0.2973887492	test 0.3029438755	bestTest 0.3028047413		total: 3.1s	remaining: 3.5s
94:	learn 0.2973887406	test 0.3029436362	bestTest 0.3028047413		total: 3.13s	remaining: 3.46s
95:	learn 0.2973887336	test 0.3029434216	bestTest 0.3028047413		total: 3.15s	remaining: 3.41s
96:	learn 0.2973874838	test 0.3029522312	bestTest 0.3028047413		total: 3.18s	remaining: 3.37s
97:	learn 0.2973874792	test 0.3029520583	bestTest 0.3028047413		total: 3.2s	remaining: 3.33s
98:	learn 0.2973874755	test 0.3029519031	bestTest 0.3028047413		total: 3.23s	remaining: 3.3s
99:	learn 0.2973874725	test 0.3029517637	bestTest 0.3028047413		total: 3.26s	remaining: 3.26s
100:	learn 0.2973874701	test 0.3029516386	bestTest 0.3028047413		

181:	learn 0.2973818338	test 0.3029987106	bestTest 0.3028047413		total: 5.63s	remaining: 557ms
182:	learn 0.2973816219	test 0.3030029921	bestTest 0.3028047413		total: 5.66s	remaining: 525ms
183:	learn 0.2973816218	test 0.303002968	bestTest 0.3028047413		total: 5.68s	remaining: 494ms
184:	learn 0.2973816217	test 0.3030029464	bestTest 0.3028047413		total: 5.72s	remaining: 464ms
185:	learn 0.2973816216	test 0.303002927	bestTest 0.3028047413		total: 5.75s	remaining: 433ms
186:	learn 0.2973816216	test 0.3030029095	bestTest 0.3028047413		total: 5.79s	remaining: 402ms
187:	learn 0.2973816215	test 0.3030028937	bestTest 0.3028047413		total: 5.82s	remaining: 372ms
188:	learn 0.2973816215	test 0.3030028795	bestTest 0.3028047413		total: 5.86s	remaining: 341ms
189:	learn 0.2973816215	test 0.3030028668	bestTest 0.3028047413		total: 5.89s	remaining: 310ms
190:	learn 0.2973816215	test 0.3030028553	bestTest 0.3028047413		total: 5.92s	remaining: 279ms
191:	learn 0.2973816214	test 0.303002845	bestTest 0.

In [8]:
def train_raise_amount_model():
    model_raise_amount_pool = ModelPool(len(names), model_dir + 'pool/raise_amount/', classifier=CatBoostRegressor)
    idx = y['private_bot_action'] == 'RAISE'
    y_raise = y[idx]['private_bot_action_amount']
    X_raise = model_raise_amount_pool.predict(X[idx])
    
    X_train, X_test, y_train, y_test = train_test_split(X_raise, y_raise, test_size=0.1, random_state=1234)
    
    model = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=2, thread_count=4,
                                   verbose=True, use_best_model=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test))
    model.save_model(model_dir + 'raise_amount.model')

train_raise_amount_model()

Borders generated
0:	learn 44.8270179	test 43.16961205	bestTest 43.16961205		total: 201ms	remaining: 40s
1:	learn 42.12766048	test 40.23505914	bestTest 40.23505914		total: 208ms	remaining: 20.6s
2:	learn 39.87720978	test 37.94315936	bestTest 37.94315936		total: 214ms	remaining: 14.1s
3:	learn 37.92583919	test 35.77102388	bestTest 35.77102388		total: 221ms	remaining: 10.8s
4:	learn 36.16187677	test 33.92669444	bestTest 33.92669444		total: 227ms	remaining: 8.84s
5:	learn 34.55005228	test 32.34221146	bestTest 32.34221146		total: 233ms	remaining: 7.54s
6:	learn 33.1672103	test 30.87821545	bestTest 30.87821545		total: 239ms	remaining: 6.59s
7:	learn 31.98968913	test 29.57901404	bestTest 29.57901404		total: 246ms	remaining: 5.9s
8:	learn 30.96585206	test 28.5086688	bestTest 28.5086688		total: 252ms	remaining: 5.34s
9:	learn 29.95510397	test 27.53550529	bestTest 27.53550529		total: 258ms	remaining: 4.9s
10:	learn 29.05890916	test 26.5488234	bestTest 26.5488234		total: 264ms	remaining: 4.54s
1

94:	learn 22.47600608	test 21.09796479	bestTest 21.04381372		total: 812ms	remaining: 898ms
95:	learn 22.46879424	test 21.10663519	bestTest 21.04381372		total: 820ms	remaining: 888ms
96:	learn 22.45783515	test 21.12791756	bestTest 21.04381372		total: 827ms	remaining: 878ms
97:	learn 22.45676115	test 21.13283702	bestTest 21.04381372		total: 835ms	remaining: 869ms
98:	learn 22.45487439	test 21.13235233	bestTest 21.04381372		total: 843ms	remaining: 860ms
99:	learn 22.44912508	test 21.14909179	bestTest 21.04381372		total: 850ms	remaining: 850ms
100:	learn 22.44614362	test 21.14991493	bestTest 21.04381372		total: 856ms	remaining: 839ms
101:	learn 22.43359575	test 21.14637334	bestTest 21.04381372		total: 862ms	remaining: 828ms
102:	learn 22.43196997	test 21.14583276	bestTest 21.04381372		total: 868ms	remaining: 818ms
103:	learn 22.43083839	test 21.14104846	bestTest 21.04381372		total: 875ms	remaining: 807ms
104:	learn 22.42824946	test 21.13617904	bestTest 21.04381372		total: 881ms	remaining: 

185:	learn 22.07823962	test 21.15151893	bestTest 21.04381372		total: 1.43s	remaining: 107ms
186:	learn 22.07775536	test 21.15109744	bestTest 21.04381372		total: 1.43s	remaining: 99.6ms
187:	learn 22.07705954	test 21.15140244	bestTest 21.04381372		total: 1.44s	remaining: 92ms
188:	learn 22.07698974	test 21.15214362	bestTest 21.04381372		total: 1.45s	remaining: 84.3ms
189:	learn 22.07670679	test 21.15203635	bestTest 21.04381372		total: 1.46s	remaining: 76.6ms
190:	learn 22.07558616	test 21.15961571	bestTest 21.04381372		total: 1.46s	remaining: 68.9ms
191:	learn 22.07529253	test 21.16309531	bestTest 21.04381372		total: 1.47s	remaining: 61.2ms
192:	learn 22.07473447	test 21.16162738	bestTest 21.04381372		total: 1.48s	remaining: 53.5ms
193:	learn 22.07431143	test 21.15890349	bestTest 21.04381372		total: 1.48s	remaining: 45.8ms
194:	learn 22.06884739	test 21.1663897	bestTest 21.04381372		total: 1.49s	remaining: 38.2ms
195:	learn 22.06335808	test 21.17392109	bestTest 21.04381372		total: 1.5s	

# TRAIN POOLS

In [3]:
def train_call_models():
    for i, name in enumerate(names):
        print(i, name)
        y_bot = y[y['private_bot_name'] == name]
        X_game = X.loc[y_bot.index]
        y_game = y_bot['private_bot_action'].replace({'FOLD': 0, 'CALL': 1, 'RAISE': 1})
        
        X_train, X_test, y_train, y_test = train_test_split(X_game, y_game, test_size=0.1, random_state=1234)

        model = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=8, thread_count=4,
                                   verbose=True, use_best_model=True)
        model.fit(X_train, y_train, eval_set=(X_test, y_test))

        model.save_model(model_dir + 'pool/call/' + str(i) + '.model')

train_call_models()

0 KillFish
Borders generated
0:	learn 0.5426269121	test 0.5462086404	bestTest 0.5462086404		total: 645ms	remaining: 2m 8s
1:	learn 0.4347995862	test 0.4399897358	bestTest 0.4399897358		total: 923ms	remaining: 1m 31s
2:	learn 0.370941392	test 0.3770314909	bestTest 0.3770314909		total: 1.13s	remaining: 1m 14s
3:	learn 0.3264892304	test 0.3341316343	bestTest 0.3341316343		total: 1.35s	remaining: 1m 6s
4:	learn 0.2933827635	test 0.3014041802	bestTest 0.3014041802		total: 1.57s	remaining: 1m 1s
5:	learn 0.2702762606	test 0.2798140824	bestTest 0.2798140824		total: 1.79s	remaining: 57.8s
6:	learn 0.2535493141	test 0.2627398809	bestTest 0.2627398809		total: 2s	remaining: 55.2s
7:	learn 0.2415610934	test 0.2501307711	bestTest 0.2501307711		total: 2.22s	remaining: 53.4s
8:	learn 0.2276692498	test 0.2368505019	bestTest 0.2368505019		total: 2.45s	remaining: 51.9s
9:	learn 0.2162436007	test 0.2256265108	bestTest 0.2256265108		total: 2.67s	remaining: 50.7s
10:	learn 0.2102600479	test 0.2194111328	be

88:	learn 0.08859332823	test 0.09890081102	bestTest 0.09890081102		total: 20.7s	remaining: 25.9s
89:	learn 0.08841857635	test 0.0987398067	bestTest 0.0987398067		total: 21s	remaining: 25.6s
90:	learn 0.08814176263	test 0.09859334359	bestTest 0.09859334359		total: 21.2s	remaining: 25.4s
91:	learn 0.0880051603	test 0.09847858659	bestTest 0.09847858659		total: 21.4s	remaining: 25.1s
92:	learn 0.08758602104	test 0.09807482524	bestTest 0.09807482524		total: 21.6s	remaining: 24.9s
93:	learn 0.08709042127	test 0.09763996735	bestTest 0.09763996735		total: 21.9s	remaining: 24.6s
94:	learn 0.0867236992	test 0.09744099212	bestTest 0.09744099212		total: 22.1s	remaining: 24.4s
95:	learn 0.08658989479	test 0.09732742855	bestTest 0.09732742855		total: 22.3s	remaining: 24.2s
96:	learn 0.08635858429	test 0.09715670864	bestTest 0.09715670864		total: 22.5s	remaining: 23.9s
97:	learn 0.08584384574	test 0.09664188028	bestTest 0.09664188028		total: 22.7s	remaining: 23.7s
98:	learn 0.08551867475	test 0.09644

173:	learn 0.06738355455	test 0.08570547418	bestTest 0.08570547418		total: 39.5s	remaining: 5.91s
174:	learn 0.06718927253	test 0.08555350153	bestTest 0.08555350153		total: 39.8s	remaining: 5.68s
175:	learn 0.06710898403	test 0.08558959249	bestTest 0.08555350153		total: 40s	remaining: 5.45s
176:	learn 0.06698876936	test 0.08548569673	bestTest 0.08548569673		total: 40.2s	remaining: 5.22s
177:	learn 0.06680911648	test 0.0854311814	bestTest 0.0854311814		total: 40.4s	remaining: 5s
178:	learn 0.06667070311	test 0.08544716415	bestTest 0.0854311814		total: 40.6s	remaining: 4.77s
179:	learn 0.06644392129	test 0.08529030452	bestTest 0.08529030452		total: 40.9s	remaining: 4.54s
180:	learn 0.06625599938	test 0.08515408948	bestTest 0.08515408948		total: 41.1s	remaining: 4.31s
181:	learn 0.06614390762	test 0.0851605011	bestTest 0.08515408948		total: 41.3s	remaining: 4.08s
182:	learn 0.06609824949	test 0.08514766725	bestTest 0.08514766725		total: 41.5s	remaining: 3.86s
183:	learn 0.06599499262	test

60:	learn 0.08064362817	test 0.1019650304	bestTest 0.1019650304		total: 9.63s	remaining: 21.9s
61:	learn 0.08033893785	test 0.1017069311	bestTest 0.1017069311		total: 9.8s	remaining: 21.8s
62:	learn 0.08009104199	test 0.1013786061	bestTest 0.1013786061		total: 9.97s	remaining: 21.7s
63:	learn 0.07982413673	test 0.101307072	bestTest 0.101307072		total: 10.1s	remaining: 21.5s
64:	learn 0.07945676557	test 0.1009896016	bestTest 0.1009896016		total: 10.3s	remaining: 21.4s
65:	learn 0.07899011546	test 0.1005784459	bestTest 0.1005784459		total: 10.4s	remaining: 21.2s
66:	learn 0.07856043264	test 0.1003806376	bestTest 0.1003806376		total: 10.6s	remaining: 21s
67:	learn 0.07832264117	test 0.1000852344	bestTest 0.1000852344		total: 10.7s	remaining: 20.9s
68:	learn 0.07707684315	test 0.09883773475	bestTest 0.09883773475		total: 10.9s	remaining: 20.7s
69:	learn 0.07689479439	test 0.09868310849	bestTest 0.09868310849		total: 11.1s	remaining: 20.5s
70:	learn 0.07647308959	test 0.09812545082	bestTest

146:	learn 0.05542995079	test 0.08345925764	bestTest 0.08345925764		total: 23.2s	remaining: 8.36s
147:	learn 0.05516284317	test 0.08324704013	bestTest 0.08324704013		total: 23.3s	remaining: 8.2s
148:	learn 0.0548955499	test 0.08328040675	bestTest 0.08324704013		total: 23.5s	remaining: 8.04s
149:	learn 0.05485264328	test 0.08325631393	bestTest 0.08324704013		total: 23.6s	remaining: 7.88s
150:	learn 0.05477572266	test 0.08323561066	bestTest 0.08323561066		total: 23.8s	remaining: 7.74s
151:	learn 0.0546458647	test 0.08310591282	bestTest 0.08310591282		total: 24s	remaining: 7.58s
152:	learn 0.05418496977	test 0.08288842203	bestTest 0.08288842203		total: 24.2s	remaining: 7.43s
153:	learn 0.05402265933	test 0.08288104133	bestTest 0.08288104133		total: 24.4s	remaining: 7.28s
154:	learn 0.05387197557	test 0.08291046679	bestTest 0.08288104133		total: 24.5s	remaining: 7.12s
155:	learn 0.05359602993	test 0.08297632897	bestTest 0.08288104133		total: 24.7s	remaining: 6.96s
156:	learn 0.05338947812	

30:	learn 0.08130248682	test 0.08358081283	bestTest 0.08358081283		total: 7.76s	remaining: 42.3s
31:	learn 0.08065214668	test 0.08309466578	bestTest 0.08309466578		total: 8.01s	remaining: 42.1s
32:	learn 0.07933666713	test 0.0818947621	bestTest 0.0818947621		total: 8.26s	remaining: 41.8s
33:	learn 0.07897840043	test 0.08157046221	bestTest 0.08157046221		total: 8.5s	remaining: 41.5s
34:	learn 0.07784194264	test 0.08047961614	bestTest 0.08047961614		total: 8.75s	remaining: 41.2s
35:	learn 0.07691972106	test 0.07966575827	bestTest 0.07966575827		total: 8.99s	remaining: 40.9s
36:	learn 0.07587714329	test 0.07871866757	bestTest 0.07871866757		total: 9.22s	remaining: 40.6s
37:	learn 0.07556053413	test 0.07847680285	bestTest 0.07847680285		total: 9.51s	remaining: 40.5s
38:	learn 0.07496634493	test 0.0778001721	bestTest 0.0778001721		total: 9.78s	remaining: 40.4s
39:	learn 0.07451775433	test 0.07754224169	bestTest 0.07754224169		total: 10.1s	remaining: 40.4s
40:	learn 0.07414361906	test 0.0772

115:	learn 0.05307612995	test 0.06140887662	bestTest 0.06140887662		total: 30.9s	remaining: 22.4s
116:	learn 0.05300921602	test 0.06130842042	bestTest 0.06130842042		total: 31.1s	remaining: 22.1s
117:	learn 0.05294029706	test 0.06129794873	bestTest 0.06129794873		total: 31.4s	remaining: 21.8s
118:	learn 0.05268855944	test 0.0609849705	bestTest 0.0609849705		total: 31.6s	remaining: 21.5s
119:	learn 0.05249402641	test 0.06094687632	bestTest 0.06094687632		total: 31.9s	remaining: 21.3s
120:	learn 0.05221957374	test 0.06086697924	bestTest 0.06086697924		total: 32.1s	remaining: 21s
121:	learn 0.05214591492	test 0.06082816259	bestTest 0.06082816259		total: 32.4s	remaining: 20.7s
122:	learn 0.05201806465	test 0.06084768593	bestTest 0.06082816259		total: 32.6s	remaining: 20.4s
123:	learn 0.05184785831	test 0.06069103298	bestTest 0.06069103298		total: 32.9s	remaining: 20.2s
124:	learn 0.05147221689	test 0.06048898301	bestTest 0.06048898301		total: 33.2s	remaining: 19.9s
125:	learn 0.0514204838	

In [4]:
def train_raise_models():
    for i, name in enumerate(names):
        print(i, name)
        y_bot = y[(y['private_bot_name'] == name) & (y['private_bot_action'] != 'FOLD')]
        X_game = X.loc[y_bot.index]
        y_game = y_bot['private_bot_action'].replace({'CALL': 0, 'RAISE': 1})
        
        X_train, X_test, y_train, y_test = train_test_split(X_game, y_game, test_size=0.1, random_state=1234)
        
        if len(y_game.value_counts()) == 1:
            y_train.iloc[0] = 0
            y_train.iloc[1] = 1
            y_test.iloc[0] = 0
            y_test.iloc[1] = 1

        model = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=8, thread_count=4,
                                   verbose=True, use_best_model=True)
        model.fit(X_train, y_train, eval_set=(X_test, y_test))

        model.save_model(model_dir + 'pool/raise/' + str(i) + '.model')

train_raise_models()

0 KillFish
Borders generated
0:	learn 0.5504781859	test 0.5487271723	bestTest 0.5487271723		total: 416ms	remaining: 1m 22s
1:	learn 0.4714719087	test 0.4692736514	bestTest 0.4692736514		total: 562ms	remaining: 55.6s
2:	learn 0.3918475617	test 0.3875398308	bestTest 0.3875398308		total: 710ms	remaining: 46.6s
3:	learn 0.3487598253	test 0.3447754899	bestTest 0.3447754899		total: 777ms	remaining: 38.1s
4:	learn 0.3136513302	test 0.3094392749	bestTest 0.3094392749		total: 940ms	remaining: 36.7s
5:	learn 0.2794405869	test 0.2753683274	bestTest 0.2753683274		total: 1.15s	remaining: 37.1s
6:	learn 0.2492262892	test 0.2447582852	bestTest 0.2447582852		total: 1.31s	remaining: 36.2s
7:	learn 0.2282450565	test 0.2239284758	bestTest 0.2239284758		total: 1.49s	remaining: 35.7s
8:	learn 0.2081956564	test 0.2040277018	bestTest 0.2040277018		total: 1.65s	remaining: 35.1s
9:	learn 0.1913919704	test 0.1873579742	bestTest 0.1873579742		total: 1.83s	remaining: 34.7s
10:	learn 0.1805818352	test 0.1752837918

89:	learn 0.08342627492	test 0.09920882077	bestTest 0.09920882077		total: 13.9s	remaining: 17s
90:	learn 0.08318738554	test 0.09901092843	bestTest 0.09901092843		total: 14.1s	remaining: 16.9s
91:	learn 0.08232001446	test 0.09877580868	bestTest 0.09877580868		total: 14.3s	remaining: 16.8s
92:	learn 0.08215178431	test 0.09891561331	bestTest 0.09877580868		total: 14.5s	remaining: 16.6s
93:	learn 0.08156627285	test 0.09875671633	bestTest 0.09875671633		total: 14.7s	remaining: 16.6s
94:	learn 0.08130778746	test 0.09868656715	bestTest 0.09868656715		total: 14.8s	remaining: 16.4s
95:	learn 0.0810059275	test 0.09865363394	bestTest 0.09865363394		total: 15s	remaining: 16.2s
96:	learn 0.08046353596	test 0.09828390302	bestTest 0.09828390302		total: 15.1s	remaining: 16.1s
97:	learn 0.07975665789	test 0.09791173324	bestTest 0.09791173324		total: 15.3s	remaining: 15.9s
98:	learn 0.07941197052	test 0.09785915129	bestTest 0.09785915129		total: 15.4s	remaining: 15.7s
99:	learn 0.07877718398	test 0.0974

175:	learn 0.05814394227	test 0.09275742056	bestTest 0.09270864887		total: 26.6s	remaining: 3.63s
176:	learn 0.05807712092	test 0.09272191398	bestTest 0.09270864887		total: 26.8s	remaining: 3.48s
177:	learn 0.05789582307	test 0.0926242896	bestTest 0.0926242896		total: 27s	remaining: 3.33s
178:	learn 0.0575691702	test 0.0923558172	bestTest 0.0923558172		total: 27.1s	remaining: 3.18s
179:	learn 0.05741340739	test 0.09242646614	bestTest 0.0923558172		total: 27.3s	remaining: 3.03s
180:	learn 0.05718125183	test 0.09225649358	bestTest 0.09225649358		total: 27.5s	remaining: 2.88s
181:	learn 0.05695391954	test 0.09211926113	bestTest 0.09211926113		total: 27.6s	remaining: 2.73s
182:	learn 0.05685047352	test 0.09210323844	bestTest 0.09210323844		total: 27.8s	remaining: 2.58s
183:	learn 0.05662996574	test 0.09200056792	bestTest 0.09200056792		total: 27.9s	remaining: 2.43s
184:	learn 0.05638520986	test 0.09192706613	bestTest 0.09192706613		total: 28s	remaining: 2.27s
185:	learn 0.05623015189	test 

61:	learn 0.1015740006	test 0.1116749158	bestTest 0.1116749158		total: 5.79s	remaining: 12.9s
62:	learn 0.101034534	test 0.1117164902	bestTest 0.1116749158		total: 5.88s	remaining: 12.8s
63:	learn 0.1007871007	test 0.1117003585	bestTest 0.1116749158		total: 5.99s	remaining: 12.7s
64:	learn 0.09999227727	test 0.1106642623	bestTest 0.1106642623		total: 6.08s	remaining: 12.6s
65:	learn 0.09951939258	test 0.1100110307	bestTest 0.1100110307		total: 6.19s	remaining: 12.6s
66:	learn 0.09913722603	test 0.1095369661	bestTest 0.1095369661		total: 6.28s	remaining: 12.5s
67:	learn 0.09851970847	test 0.1092150826	bestTest 0.1092150826		total: 6.4s	remaining: 12.4s
68:	learn 0.09779549967	test 0.109059055	bestTest 0.109059055		total: 6.49s	remaining: 12.3s
69:	learn 0.09748277597	test 0.1087921044	bestTest 0.1087921044		total: 6.58s	remaining: 12.2s
70:	learn 0.09668672708	test 0.1084787256	bestTest 0.1084787256		total: 6.67s	remaining: 12.1s
71:	learn 0.09648244213	test 0.1085256772	bestTest 0.1084

148:	learn 0.06605638524	test 0.09562578698	bestTest 0.09551622228		total: 13.9s	remaining: 4.76s
149:	learn 0.0658972291	test 0.09554050207	bestTest 0.09551622228		total: 14s	remaining: 4.67s
150:	learn 0.06549672179	test 0.09524924236	bestTest 0.09524924236		total: 14.1s	remaining: 4.59s
151:	learn 0.06521544748	test 0.09506567969	bestTest 0.09506567969		total: 14.2s	remaining: 4.5s
152:	learn 0.06506311954	test 0.09507037784	bestTest 0.09506567969		total: 14.3s	remaining: 4.41s
153:	learn 0.06478747651	test 0.09516112392	bestTest 0.09506567969		total: 14.5s	remaining: 4.32s
154:	learn 0.06463320975	test 0.09520733366	bestTest 0.09506567969		total: 14.6s	remaining: 4.23s
155:	learn 0.06455562021	test 0.0952365219	bestTest 0.09506567969		total: 14.7s	remaining: 4.13s
156:	learn 0.06402740727	test 0.094991463	bestTest 0.094991463		total: 14.7s	remaining: 4.04s
157:	learn 0.06345953865	test 0.0946770967	bestTest 0.0946770967		total: 14.8s	remaining: 3.94s
158:	learn 0.06308813151	test 0

33:	learn 0.2290060291	test 0.251320451	bestTest 0.251320451		total: 7.87s	remaining: 38.4s
34:	learn 0.2258740723	test 0.2481136631	bestTest 0.2481136631		total: 8.12s	remaining: 38.3s
35:	learn 0.2234946111	test 0.245587025	bestTest 0.245587025		total: 8.36s	remaining: 38.1s
36:	learn 0.2199557208	test 0.2424665874	bestTest 0.2424665874		total: 8.59s	remaining: 37.8s
37:	learn 0.2180336875	test 0.2405903468	bestTest 0.2405903468		total: 8.79s	remaining: 37.5s
38:	learn 0.2170611038	test 0.2398028643	bestTest 0.2398028643		total: 9s	remaining: 37.2s
39:	learn 0.2152272609	test 0.2384111649	bestTest 0.2384111649		total: 9.21s	remaining: 36.9s
40:	learn 0.2126801877	test 0.2358769253	bestTest 0.2358769253		total: 9.42s	remaining: 36.5s
41:	learn 0.2122895198	test 0.2356475585	bestTest 0.2356475585		total: 9.64s	remaining: 36.3s
42:	learn 0.2102695599	test 0.2340215045	bestTest 0.2340215045		total: 9.85s	remaining: 36s
43:	learn 0.2092011273	test 0.2331579031	bestTest 0.2331579031		total

121:	learn 0.161676135	test 0.198708374	bestTest 0.1986996314		total: 27.1s	remaining: 17.4s
122:	learn 0.1612650877	test 0.1986024283	bestTest 0.1986024283		total: 27.3s	remaining: 17.1s
123:	learn 0.1602845235	test 0.1977018718	bestTest 0.1977018718		total: 27.6s	remaining: 16.9s
124:	learn 0.1600607299	test 0.1976534012	bestTest 0.1976534012		total: 27.8s	remaining: 16.7s
125:	learn 0.1598595072	test 0.1976767057	bestTest 0.1976534012		total: 28s	remaining: 16.4s
126:	learn 0.1594562385	test 0.1974197204	bestTest 0.1974197204		total: 28.2s	remaining: 16.2s
127:	learn 0.1592203701	test 0.1972108882	bestTest 0.1972108882		total: 28.4s	remaining: 16s
128:	learn 0.1580689001	test 0.1961867054	bestTest 0.1961867054		total: 28.6s	remaining: 15.8s
129:	learn 0.157870501	test 0.1962036417	bestTest 0.1961867054		total: 28.8s	remaining: 15.5s
130:	learn 0.1566635684	test 0.1949727122	bestTest 0.1949727122		total: 29.1s	remaining: 15.3s
131:	learn 0.1562983396	test 0.1947625944	bestTest 0.1947

In [5]:
def train_raise_amount_models():
    for i, name in enumerate(names):
        print(i, name)
        y_bot = y[(y['private_bot_name'] == name) & (y['private_bot_action'] == 'RAISE')]
        X_game = X.loc[y_bot.index]
        y_game = y_bot['private_bot_action_amount']

        X_train, X_test, y_train, y_test = train_test_split(X_game, y_game, test_size=0.1, random_state=1234)

        model = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=8, thread_count=4,
                                   verbose=True, use_best_model=True)
        model.fit(X_train, y_train, eval_set=(X_test, y_test))
        
        model.save_model(model_dir + 'pool/raise_amount/' + str(i) + '.model')

train_raise_amount_models()

0 KillFish
Borders generated
0:	learn 60.15636327	test 42.54547044	bestTest 42.54547044		total: 185ms	remaining: 36.8s
1:	learn 56.59435813	test 40.25687128	bestTest 40.25687128		total: 234ms	remaining: 23.1s
2:	learn 53.56823052	test 37.82595328	bestTest 37.82595328		total: 279ms	remaining: 18.3s
3:	learn 50.8064362	test 35.55745685	bestTest 35.55745685		total: 322ms	remaining: 15.8s
4:	learn 48.10862081	test 33.64006722	bestTest 33.64006722		total: 372ms	remaining: 14.5s
5:	learn 45.73109762	test 32.08203245	bestTest 32.08203245		total: 417ms	remaining: 13.5s
6:	learn 43.79105232	test 30.68724545	bestTest 30.68724545		total: 461ms	remaining: 12.7s
7:	learn 42.08331214	test 29.54899438	bestTest 29.54899438		total: 493ms	remaining: 11.8s
8:	learn 40.50121226	test 28.58086778	bestTest 28.58086778		total: 536ms	remaining: 11.4s
9:	learn 39.08526578	test 27.73432123	bestTest 27.73432123		total: 580ms	remaining: 11s
10:	learn 37.80407517	test 27.06047181	bestTest 27.06047181		total: 632ms	

92:	learn 22.24821133	test 24.93792544	bestTest 24.37936109		total: 5.47s	remaining: 6.29s
93:	learn 22.23004811	test 24.98698126	bestTest 24.37936109		total: 5.51s	remaining: 6.22s
94:	learn 22.05249278	test 25.04843408	bestTest 24.37936109		total: 5.56s	remaining: 6.14s
95:	learn 21.97854646	test 25.05375101	bestTest 24.37936109		total: 5.6s	remaining: 6.07s
96:	learn 21.77695175	test 25.08278129	bestTest 24.37936109		total: 5.65s	remaining: 6s
97:	learn 21.73920894	test 25.16805513	bestTest 24.37936109		total: 5.7s	remaining: 5.93s
98:	learn 21.70304481	test 25.18388522	bestTest 24.37936109		total: 5.75s	remaining: 5.87s
99:	learn 21.6379735	test 25.17737195	bestTest 24.37936109		total: 5.79s	remaining: 5.79s
100:	learn 21.53447907	test 25.12957019	bestTest 24.37936109		total: 5.84s	remaining: 5.72s
101:	learn 21.51742406	test 25.16661755	bestTest 24.37936109		total: 5.88s	remaining: 5.65s
102:	learn 21.47112192	test 25.16129095	bestTest 24.37936109		total: 5.94s	remaining: 5.59s
10

184:	learn 16.4730452	test 25.26843883	bestTest 24.37936109		total: 9.82s	remaining: 796ms
185:	learn 16.39206758	test 25.3023626	bestTest 24.37936109		total: 9.86s	remaining: 742ms
186:	learn 16.29792012	test 25.37250849	bestTest 24.37936109		total: 9.91s	remaining: 689ms
187:	learn 16.28374712	test 25.37903864	bestTest 24.37936109		total: 9.95s	remaining: 635ms
188:	learn 16.22102894	test 25.38090009	bestTest 24.37936109		total: 10s	remaining: 582ms
189:	learn 16.09970554	test 25.38417524	bestTest 24.37936109		total: 10s	remaining: 529ms
190:	learn 16.09100077	test 25.36834841	bestTest 24.37936109		total: 10.1s	remaining: 475ms
191:	learn 16.07921732	test 25.36304301	bestTest 24.37936109		total: 10.1s	remaining: 422ms
192:	learn 16.02996205	test 25.41366941	bestTest 24.37936109		total: 10.2s	remaining: 369ms
193:	learn 16.02322171	test 25.41390297	bestTest 24.37936109		total: 10.2s	remaining: 316ms
194:	learn 15.85266888	test 25.42466116	bestTest 24.37936109		total: 10.3s	remaining: 

78:	learn 11.26043508	test 18.83715731	bestTest 16.96593126		total: 3.56s	remaining: 5.45s
79:	learn 11.0579132	test 18.86506356	bestTest 16.96593126		total: 3.6s	remaining: 5.4s
80:	learn 11.04671875	test 18.8648695	bestTest 16.96593126		total: 3.64s	remaining: 5.35s
81:	learn 10.99067689	test 18.89259959	bestTest 16.96593126		total: 3.69s	remaining: 5.3s
82:	learn 10.92244821	test 18.91206613	bestTest 16.96593126		total: 3.73s	remaining: 5.25s
83:	learn 10.88042395	test 18.95120961	bestTest 16.96593126		total: 3.77s	remaining: 5.21s
84:	learn 10.80584334	test 18.9089876	bestTest 16.96593126		total: 3.82s	remaining: 5.16s
85:	learn 10.77105364	test 18.86691377	bestTest 16.96593126		total: 3.86s	remaining: 5.11s
86:	learn 10.64510113	test 18.88943142	bestTest 16.96593126		total: 3.9s	remaining: 5.07s
87:	learn 10.5830235	test 18.89650338	bestTest 16.96593126		total: 3.95s	remaining: 5.02s
88:	learn 10.55898772	test 18.97886645	bestTest 16.96593126		total: 3.99s	remaining: 4.98s
89:	lea

169:	learn 8.428566538	test 18.47029905	bestTest 16.96593126		total: 8.96s	remaining: 1.58s
170:	learn 8.407599681	test 18.47044364	bestTest 16.96593126		total: 9.02s	remaining: 1.53s
171:	learn 8.375465306	test 18.36356528	bestTest 16.96593126		total: 9.07s	remaining: 1.48s
172:	learn 8.316750152	test 18.37745804	bestTest 16.96593126		total: 9.12s	remaining: 1.42s
173:	learn 8.310547478	test 18.3746935	bestTest 16.96593126		total: 9.17s	remaining: 1.37s
174:	learn 8.289692748	test 18.37599165	bestTest 16.96593126		total: 9.22s	remaining: 1.32s
175:	learn 8.280971807	test 18.3765627	bestTest 16.96593126		total: 9.27s	remaining: 1.26s
176:	learn 8.262569252	test 18.3687778	bestTest 16.96593126		total: 9.31s	remaining: 1.21s
177:	learn 8.232031254	test 18.37571281	bestTest 16.96593126		total: 9.35s	remaining: 1.16s
178:	learn 8.227333185	test 18.37201139	bestTest 16.96593126		total: 9.4s	remaining: 1.1s
179:	learn 8.221545044	test 18.3703419	bestTest 16.96593126		total: 9.44s	remaining: 

59:	learn 10.14919925	test 18.46588145	bestTest 18.46588145		total: 6.32s	remaining: 14.7s
60:	learn 10.1197831	test 18.47220739	bestTest 18.46588145		total: 6.45s	remaining: 14.7s
61:	learn 10.05916738	test 18.48719337	bestTest 18.46588145		total: 6.58s	remaining: 14.6s
62:	learn 10.02471799	test 18.54353299	bestTest 18.46588145		total: 6.66s	remaining: 14.5s
63:	learn 9.967250385	test 18.53605087	bestTest 18.46588145		total: 6.9s	remaining: 14.7s
64:	learn 9.939911821	test 18.48526323	bestTest 18.46588145		total: 7s	remaining: 14.5s
65:	learn 9.904305367	test 18.4709501	bestTest 18.46588145		total: 7.09s	remaining: 14.4s
66:	learn 9.86017444	test 18.47987928	bestTest 18.46588145		total: 7.2s	remaining: 14.3s
67:	learn 9.843501917	test 18.45703977	bestTest 18.45703977		total: 7.42s	remaining: 14.4s
68:	learn 9.783985316	test 18.43862734	bestTest 18.43862734		total: 7.54s	remaining: 14.3s
69:	learn 9.729707567	test 18.42381964	bestTest 18.42381964		total: 7.62s	remaining: 14.2s
70:	lea

150:	learn 7.292539346	test 18.1127173	bestTest 18.1127173		total: 16.5s	remaining: 5.35s
151:	learn 7.258120871	test 18.11369958	bestTest 18.1127173		total: 16.6s	remaining: 5.24s
152:	learn 7.249622721	test 18.10850054	bestTest 18.10850054		total: 16.7s	remaining: 5.12s
153:	learn 7.217483929	test 18.14151666	bestTest 18.10850054		total: 16.8s	remaining: 5.01s
154:	learn 7.204188526	test 18.14470946	bestTest 18.10850054		total: 16.9s	remaining: 4.89s
155:	learn 7.178193446	test 18.09660927	bestTest 18.09660927		total: 16.9s	remaining: 4.78s
156:	learn 7.17348652	test 18.09477502	bestTest 18.09477502		total: 17s	remaining: 4.67s
157:	learn 7.165686512	test 18.10122382	bestTest 18.09477502		total: 17.1s	remaining: 4.56s
158:	learn 7.164471179	test 18.10132315	bestTest 18.09477502		total: 17.2s	remaining: 4.45s
159:	learn 7.130437153	test 18.11948242	bestTest 18.09477502		total: 17.3s	remaining: 4.33s
160:	learn 7.125068979	test 18.1362792	bestTest 18.09477502		total: 17.4s	remaining: 4